<a href="https://colab.research.google.com/github/AmrRagab0/-SAAT-Semi-Automated-Annotation-Technique-/blob/main/Semi_automated_annotation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code runs on one match
We need to create a pipeline that goes as follows: <br>
* For each match <br>
  1- if it's not in a directory , mkdir and put the match in it <br>
  2- convert the video to audio (code availabe below)<br>
  3- use whisper model to detect language and transcribe it (code availabe below)<br>
  4- save the transcription file (vtt or srt, whatever is best for search)<br>
  5- create a program that extracts a set of keywords from the transcription file, and then saves those keywords timings in a new file with the name of the action as title (e.g foul,goal..)<br>



This is just to hide annoying cell output
just put %%capture at the start of the cell that you don't want to show the output of

In [ ]:
!pip install -q gwpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.2 MB/s eta 0:00:00


In [ ]:
%%capture
!pip install git+https://github.com/openai/whisper.git

In [ ]:
import io
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import urllib
import tarfile
import whisper
import torchaudio

from scipy.io import wavfile
from tqdm.notebook import tqdm


pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = whisper.load_model("small")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|███████████████████████████████████████| 461M/461M [00:07<00:00, 65.7MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Model is multilingual and has 240,582,912 parameters.


In [ ]:
!sudo apt install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


# Downloading the matches


## From Youtube

In [ ]:
!pip install yt-dlp
!pip install youtube-dl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.2 MB/s eta 0:00:00


## To download, you may want to upload a cookie file

In [ ]:
#https://github.com/ytdl-org/youtube-dl/blob/master/README.md#embedding-youtube-dl

import yt_dlp

video_url = 'https://www.youtube.com/watch?v=OFbyNU6UQQs'

ydl_opts = {
    'cookiefile': '/content/youtube.com_cookies.txt',
    'outtmpl': '%(title)s.%(ext)s',
    'format': 'best'
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=OFbyNU6UQQs
[youtube] OFbyNU6UQQs: Downloading webpage
[youtube] OFbyNU6UQQs: Downloading tv client config
[youtube] OFbyNU6UQQs: Downloading player af7f576f
[youtube] OFbyNU6UQQs: Downloading tv player API JSON
[info] OFbyNU6UQQs: Downloading 1 format(s): 18
[download] Destination: FULL MATCH： Portugal v Spain ｜ 2018 FIFA World Cup.mp4
[download] 100% of  495.10MiB in 00:00:46 at 10.68MiB/s  


In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.4 MB/s eta 0:00:00
Using cached setuptools-75.8.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libtorrent-rasterbar

SystemError: initialization of libtorrent raised unreported exception

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

In [ ]:
downloads = []

In [ ]:
params = {"save_path": "/content/"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000000),
                    "MB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Convert **Video** to **audio**

In [ ]:
!ffmpeg -i "/content/FULL MATCH： Portugal v Spain ｜ 2018 FIFA World Cup.mp4" -ab 160k -ar 44100 -acodec pcm_s16le -vn fullmatch_pcm.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
#!whisper --help

# Speech to text

In [ ]:
!whisper fullmatch_pcm.wav --model small --language en

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
[00:00.000 --

## Searching and saving the keywords timestamps

In [ ]:
!pip install win32com.client

# Convert whisper output to formatted annotations

In [ ]:
import re
import json

# Match metadata (adjust these values as per your data source)
metadata = {
    "UrlLocal": "spain_laliga/2016-2017/2017-04-23 - 21-45 Real Madrid 2 - 3 Barcelona/",
    "UrlYoutube": "",
    "gameHomeTeam": "Real Madrid",
    "gameAwayTeam": "Barcelona",
    "gameDate": "23/04/2017 - 21:45",
    "gameScore": "2 - 3"
}

# Define the list of keywords to search for, categorized by action type
keywords = {
    "penalty": ["penalty", "spot kick", "foul in the box"],
    "kick_off": ["kickoff", "start of play", "restarts play"],
    "goal": ["goal", "scores", "netted", "header", "brace", "hat-trick"],
    "substitution": ["substitute", "player change", "replacement", "comes off"],
    "offside": ["offside", "flagged offside", "caught offside"],
    "shots_on_target": ["shot on target", "save", "on goal", "keeper saves","shot"],
    "shots_off_target": ["shot wide", "misses", "off target", "over the bar","shot"],
    "clearance": ["clearance", "defensive clearance", "kicked away"],
    "ball_out_of_play": ["out of play", "throw-in", "ball goes out"],
    "throw_in": ["throw-in", "throws it in","throw"],
    "foul": ["foul", "reckless tackle", "dangerous play","challenge"],
    "indirect_free_kick": ["indirect free kick", "indirect kick"],
    "direct_free_kick": ["direct free kick", "free kick", "set piece"],
    "corner": ["corner kick", "corner", "takes a corner"],
    "yellow_card": ["yellow card", "booked", "caution"],
    "red_card": ["red card", "sent off", "dismissed"],
    "yellow_to_red_card": ["second yellow card", "yellow to red"],
    "unknown": ["unspecified", "unclear"]
}

# Open the .vtt file and read its contents
with open("fullmatch_pcm.vtt", "r") as f:
    contents = f.read()

# Use regular expressions to extract the timestamp and text for each line of dialogue
pattern = r"(\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}\.\d{3})\n(.+?)\n\n"
matches = re.findall(pattern, contents)

annotations = []
position_counter = 0  # Simulated position for demonstration purposes

# Iterate over each line of dialogue and check if it contains any of the keywords
for match in matches:
    start_time, end_time, text = match
    lower_text = text.lower()  # Convert text to lowercase for case-insensitive matching
    for category, words in keywords.items():
        for word in words:
            if word in lower_text:
                annotation = {
                    "gameTime": f"2 - {start_time}",  # Assuming second half commentary for this example
                    "label": word.capitalize(),
                    "position": str(position_counter),  # Example position value
                    "team": "home" if "real madrid" in lower_text else "away",
                    "visibility": "visible"
                }
                annotations.append(annotation)
                position_counter += 1000  # Increment position for demonstration
                break  # Stop after finding a keyword in this category

# Compile the final JSON structure
output_data = {
    **metadata,
    "annotations": annotations
}

# Write the annotations to a JSON file
with open("annotations_output.json", "w") as json_file:
    json.dump(output_data, json_file, indent=4)

print("Annotations successfully saved to annotations_output.json")


Annotations successfully saved to annotations_output.json


**Filter Duplicate Soccer Events Within a 30-Second Window**

In [ ]:
import json
import re
# Function to convert gameTime to total seconds
def convert_to_seconds(game_time):
      match = re.match(r"(\d+) - (\d+):(\d+)", game_time)
      if match:
          half, minutes, seconds = map(int, match.groups())
          total_seconds = 0 + minutes * 60 + seconds
          return total_seconds
      else:
          return None

# Function to filter duplicate events within a 10-second window
def filter_annotations(annotations):
    filtered_annotations = []
    last_event_time = {}

    for annotation in annotations:
        label = annotation["label"]
        current_time = convert_to_seconds(annotation["gameTime"])

        # Check if this label has been seen before
        if label in last_event_time:
            # If the current event is within 10 seconds of the last one, skip it
            if current_time - last_event_time[label] < 30:
                continue

        # Update the last event time for this label
        last_event_time[label] = current_time
        # Add the annotation to the filtered list
        filtered_annotations.append(annotation)

    return filtered_annotations

# Read the JSON file
input_file_path = 'annotations_output.json'
with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Filter the annotations
data["annotations"] = filter_annotations(data["annotations"])

# Save the modified data to a new JSON file
output_file_path = 'annotations_output.json'
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=4)

print(f"Filtered annotations saved to {output_file_path}")

In [ ]:
!whisper -help

# Event Extraction from Soccer Commentaries Using Open-Source Hugging Face Models
Explore our notebook for extracting key soccer match events (e.g., goals, fouls, penalties) from commentaries using state-of-the-art open-source models from Hugging Face.
[Notebook](https://colab.research.google.com/drive/1K7nFs__cTw9iMswJuGHDlte6njRF-pDi?usp=sharing)
